### Libraries needed

pygbif is split up into modules for each of the major groups of API methods.

Registry - Datasets, Nodes, Installations, Networks, Organizations
Species - Taxonomic names
Occurrences - Occurrence data, including the download API
Maps - Maps, get raster maps from GBIF as png or mvt

In [1]:
import pygbif
from pygbif import occurrences as occ
from pygbif import species

import requests

import pandas as pd

### Request data from de API

#### To request data from the server:

In [2]:
url = 'http://api.gbif.org/v1/species/match?verbose=true&name='
data = requests.get(url)
data.status_code

200

#### In order to call the species with the GBIF API, it is necessary to enter the codes for each species. As it is impossible to know the codes, I am going to generate a function that, giving the scientific name, will return the codes for each one of them.

In [3]:
splist = ['Apus affinis', 'Apus caffer', 'Parnasius apollo', 'Calotriton arnoldi', 'Gallotia simonyi', 'Alytes dickhilleni',
'Meliataea cinxia', 'Silene ciliata', 'Abies pinsapo', 'Lanius minor', 'Ciconia ciconia']

def get_species_name_from_codes (sp):
    keys = [species.name_backbone(x)['usageKey'] for x in splist ]
    species_codes = dict(zip(splist, keys))
    return species_codes    

In [4]:
code_number = get_species_name_from_codes(splist)
code_number

{'Apus affinis': 5228662,
 'Apus caffer': 5228644,
 'Parnasius apollo': 1938810,
 'Calotriton arnoldi': 2431827,
 'Gallotia simonyi': 2468673,
 'Alytes dickhilleni': 2426613,
 'Meliataea cinxia': 8168644,
 'Silene ciliata': 7267056,
 'Abies pinsapo': 2685464,
 'Lanius minor': 2492854,
 'Ciconia ciconia': 2481912}

#### To get the total number of occurrences for each species
We can use the function occ.search which search for GBIF occurences.

In [5]:
response = requests.get('http://api.gbif.org/v1/occurrence/counts/datasets')
response.status_code

200

In [6]:
code_number

{'Apus affinis': 5228662,
 'Apus caffer': 5228644,
 'Parnasius apollo': 1938810,
 'Calotriton arnoldi': 2431827,
 'Gallotia simonyi': 2468673,
 'Alytes dickhilleni': 2426613,
 'Meliataea cinxia': 8168644,
 'Silene ciliata': 7267056,
 'Abies pinsapo': 2685464,
 'Lanius minor': 2492854,
 'Ciconia ciconia': 2481912}

In [120]:
def get_occurences(sp):
    
    years = range(1970, 2020)
    x = []
    
    for y in years:
        data = occ.search(taxonKey = sp, limit = 300, country = 'ES', year = str(y))
        #Get the number of occurrences:
        x.append({str(y): data['count']})
        
        # Set all the data in an unique dictionary
        final = {}
        for d in x:
            for k in d.keys():
                final[k] = final.get(k,0) + d[k]
       
    return final

In [121]:
def get_taxonomic_info(sp):
    data = occ.search(taxonKey = sp, limit = 300, country = 'ES', year = '2016')
    taxonomic = data["results"]
    for dictionary in taxonomic:
        species = dictionary["scientificName"]
        kingdom = dictionary["kingdom"] 
        genus = dictionary['genus']
        family = dictionary['family']           
        country = dictionary["country"]
        records = dictionary['basisOfRecord']
        pub = dictionary['publishingCountry']
            
        
    
    species_info = {'species': species,
                    'kingdom': kingdom,
                    'Genus': genus, 
                    'Family': family,
                    'country': country,
                    'records': records,
                    'Publishing_country': pub
               }
    return species_info

In [128]:
def join_occurrences_taxonomic(sp):
    '''
    Receive taxonomic information dictionary and dictictionry with occurences/year
    
    Args:
    
    
    Returns
        jsdf
    '''
    # occurrence dictionary to dataframe
    df_occurrences = pd.DataFrame.from_dict(data = get_occurences(sp),  orient='index').reset_index()
    df_occurrences.columns = ['Year','Occurrences']
    
    #taxonomic dictionary to dataframe
    df_taxonomic = pd.DataFrame(data = get_taxonomic_info(sp), index = [0])
    df_taxonomic = pd.concat([df_taxonomic]*550, ignore_index=True)
    
    # concatenate occurences with taxonomic information
    sp1 = pd.concat([df_occurrences,df_taxonomic], axis=1)
    
    
    return sp1

In [119]:
silene = join_occurrences_taxonomic(7267056)
silene[:5]

,Year,Occurrences,species,kingdom,Genus,Family,country,records,Publishing_country
0,1970,6,Silene ciliata Pourr.,Plantae,Silene,Caryophyllaceae,Spain,HUMAN_OBSERVATION,FR
1,1971,1,Silene ciliata Pourr.,Plantae,Silene,Caryophyllaceae,Spain,HUMAN_OBSERVATION,FR
2,1972,5,Silene ciliata Pourr.,Plantae,Silene,Caryophyllaceae,Spain,HUMAN_OBSERVATION,FR
3,1973,9,Silene ciliata Pourr.,Plantae,Silene,Caryophyllaceae,Spain,HUMAN_OBSERVATION,FR
4,1974,6,Silene ciliata Pourr.,Plantae,Silene,Caryophyllaceae,Spain,HUMAN_OBSERVATION,FR


In [129]:
%%time
for key, values in code_number.items():
    data_species = join_occurrences_taxonomic(values)

CPU times: user 12.5 s, sys: 2.27 s, total: 14.8 s
Wall time: 9min 55s


In [130]:
data_species.shape

(550, 9)

In [132]:
data_species[:200]

,Year,Occurrences,species,kingdom,Genus,Family,country,records,Publishing_country
0,1970,23.0,"Ciconia ciconia (Linnaeus, 1758)",Animalia,Ciconia,Ciconiidae,Spain,HUMAN_OBSERVATION,ES
1,1971,13.0,"Ciconia ciconia (Linnaeus, 1758)",Animalia,Ciconia,Ciconiidae,Spain,HUMAN_OBSERVATION,ES
2,1972,33.0,"Ciconia ciconia (Linnaeus, 1758)",Animalia,Ciconia,Ciconiidae,Spain,HUMAN_OBSERVATION,ES
3,1973,36.0,"Ciconia ciconia (Linnaeus, 1758)",Animalia,Ciconia,Ciconiidae,Spain,HUMAN_OBSERVATION,ES
4,1974,51.0,"Ciconia ciconia (Linnaeus, 1758)",Animalia,Ciconia,Ciconiidae,Spain,HUMAN_OBSERVATION,ES
...,...,...,...,...,...,...,...,...,...
195,NaN,NaN,"Ciconia ciconia (Linnaeus, 1758)",Animalia,Ciconia,Ciconiidae,Spain,HUMAN_OBSERVATION,ES
196,NaN,NaN,"Ciconia ciconia (Linnaeus, 1758)",Animalia,Ciconia,Ciconiidae,Spain,HUMAN_OBSERVATION,ES
197,NaN,NaN,"Ciconia ciconia (Linnaeus, 1758)",Animalia,Ciconia,Ciconiidae,Spain,HUMAN_OBSERVATION,ES
198,NaN,NaN,"Ciconia ciconia (Linnaeus, 1758)",Animalia,Ciconia,Ciconiidae,Spain,HUMAN_OBSERVATION,ES


In [133]:
data_species.to_csv("Pru_sp.csv")